In [1]:
import time
from datasets import DatasetDict, Dataset
import json
import torch
from transformers import T5Tokenizer, DataCollatorForSeq2Seq
from transformers import T5ForConditionalGeneration, Seq2SeqTrainingArguments, Seq2SeqTrainer
import nltk
import evaluate
import numpy as np

In [2]:
# 检查是否有可用的GPU
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

Using device: cuda


In [3]:
# Load the TLQA data
def read_QA_data(filepath):
    with open(filepath, 'r', encoding='utf-8') as f:
        data = json.load(f)
    return data
# Filter data to only include 'question' and 'answers'
def filter_data(data):
    filtered_data = [{"question": item["question"], "answers": item["answers"]} for item in data]
    return filtered_data
train_data = filter_data(read_QA_data('train_TLQA.json'))
val_data = filter_data(read_QA_data('val_TLQA.json'))
test_data = filter_data(read_QA_data('test_TLQA.json'))

In [4]:
# Print sample data
print("Sample training data:", train_data[0])
print("Sample validation data:", val_data[0])
print("Sample test data:", test_data[0])

Sample training data: {'question': 'List all entities that owned Radisson Hotel Group, also known as Radisson Hospitality, Inc., from 2010 to 2020.', 'answers': ['Carlson (2010, 2011, 2012, 2013, 2014, 2015, 2016)', 'HNA Group (2016, 2017, 2018)', 'Jinjiang International (2018, 2019, 2020)']}
Sample validation data: {'question': 'List all chairpersons of Christian Democratic and Flemish, also known as CD&V, from 2010 to 2020', 'answers': ['Wouter Beke (2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019)', 'Joachim Coens (2019, 2020)']}
Sample test data: {'question': 'List all sports teams Anthony Grant, also known as Anthony Paul Shaun Andrew Daure Grant, played for from 2010 to 2020.', 'answers': ['Southend United F.C. (2010, 2011, 2012)', 'Stevenage F.C. (2012, 2013)', 'Crewe Alexandra F.C. (2013, 2014, 2015)', 'Port Vale F.C. (2015, 2016, 2017, 2018, 2019, 2020)']}


In [5]:
def merge_answers(data):
    """合并每个问题的答案到一个字符串中"""
    merged_data = []
    for item in data:
        merged_answer = ', '.join(item['answers'])
        merged_item = {
            'question': item['question'],
            'answer': merged_answer
        }
        merged_data.append(merged_item)
    return merged_data

train_data = merge_answers(filter_data(read_QA_data('train_TLQA.json')))
val_data = merge_answers(filter_data(read_QA_data('val_TLQA.json')))
test_data = merge_answers(filter_data(read_QA_data('test_TLQA.json')))

In [6]:
print(test_data[0])

{'question': 'List all sports teams Anthony Grant, also known as Anthony Paul Shaun Andrew Daure Grant, played for from 2010 to 2020.', 'answer': 'Southend United F.C. (2010, 2011, 2012), Stevenage F.C. (2012, 2013), Crewe Alexandra F.C. (2013, 2014, 2015), Port Vale F.C. (2015, 2016, 2017, 2018, 2019, 2020)'}


In [7]:
model = T5ForConditionalGeneration.from_pretrained("google/flan-t5-base")
# 将模型移动到GPU
model.to(device)

T5ForConditionalGeneration(
  (shared): Embedding(32128, 768)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 768)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=768, out_features=768, bias=False)
              (k): Linear(in_features=768, out_features=768, bias=False)
              (v): Linear(in_features=768, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseGatedActDense(
              (wi_0): Linear(in_features=768, out_features=2048, bias=False)
              (wi_1): Linear(in_features=768, out_features=2048, bias=False)
              (wo):

In [8]:
pip install transformers datasets torch

Note: you may need to restart the kernel to use updated packages.


In [9]:
# Convert list of dictionaries to dictionary of lists
def convert_to_dict(data):
    keys = data[0].keys()
    return {key: [d[key] for d in data] for key in keys}
# Convert data to Hugging Face Dataset
train_dataset = Dataset.from_dict(convert_to_dict(train_data))
val_dataset = Dataset.from_dict(convert_to_dict(val_data))
test_dataset = Dataset.from_dict(convert_to_dict(test_data))
# Combine into a DatasetDict
dataset = DatasetDict({
    'train': train_dataset,
    'validation': val_dataset,
    'test': test_dataset
})
dataset

DatasetDict({
    train: Dataset({
        features: ['question', 'answer'],
        num_rows: 3212
    })
    validation: Dataset({
        features: ['question', 'answer'],
        num_rows: 1428
    })
    test: Dataset({
        features: ['question', 'answer'],
        num_rows: 1071
    })
})

In [10]:
tokenizer = T5Tokenizer.from_pretrained('google/flan-t5-large')

def preprocess_function(examples):
    # Assuming 'examples' is a batch from a dataset, 'questions' and 'answers' are expected to be lists
    questions = examples["question"]
    answers = examples["answer"]

    # Tokenize questions and answers
    # Ensure that padding is not applied yet, it will be applied batch-wise later if using a DataLoader
    model_inputs = tokenizer(questions, max_length=128, truncation=True, padding="max_length")
    labels = tokenizer(answers, max_length=128, truncation=True, padding="max_length")

    # Add labels to the model inputs, labels should be the token ids of the answers
    model_inputs['labels'] = labels['input_ids']

    return model_inputs

# Process datasets
tokenized_dataset = dataset.map(preprocess_function, batched=True, batch_size=16)


tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

D:\Anaconda3\lib\site-packages\huggingface_hub\file_download.py:157: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\chenwanxin\.cache\huggingface\hub\models--google--flan-t5-large. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expect

Map:   0%|          | 0/3212 [00:00<?, ? examples/s]

Map:   0%|          | 0/1428 [00:00<?, ? examples/s]

Map:   0%|          | 0/1071 [00:00<?, ? examples/s]

In [11]:
print(tokenized_dataset['train'][0])

{'question': 'List all entities that owned Radisson Hotel Group, also known as Radisson Hospitality, Inc., from 2010 to 2020.', 'answer': 'Carlson (2010, 2011, 2012, 2013, 2014, 2015, 2016), HNA Group (2016, 2017, 2018), Jinjiang International (2018, 2019, 2020)', 'input_ids': [6792, 66, 12311, 24, 4157, 6324, 12661, 2282, 1531, 6, 92, 801, 38, 6324, 12661, 4457, 485, 6, 1542, 5, 6, 45, 2735, 12, 6503, 5, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

In [12]:
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)
# Global Parameters
L_RATE = 3e-4
BATCH_SIZE = 8
PER_DEVICE_EVAL_BATCH = 4
WEIGHT_DECAY = 0.01
SAVE_TOTAL_LIM = 3
NUM_EPOCHS = 5


# Set up training arguments
training_args = Seq2SeqTrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    logging_steps=100,  # 每100步记录一次日志
    learning_rate=L_RATE,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=PER_DEVICE_EVAL_BATCH,
    weight_decay=WEIGHT_DECAY,
    save_total_limit=SAVE_TOTAL_LIM,
    num_train_epochs=NUM_EPOCHS,
    predict_with_generate=True,
    push_to_hub=False
)

# Set up trainer
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    #compute_metrics=compute_metrics
)
     

D:\Anaconda3\lib\site-packages\transformers\training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [13]:
# Trigger the model training
trainer.train()

Epoch,Training Loss,Validation Loss
1,0.478700,0.402435
2,0.394300,0.386627
3,0.358300,0.380146
4,0.321700,0.380816
5,0.304500,0.384649


TrainOutput(global_step=2010, training_loss=0.4632857378442489, metrics={'train_runtime': 1314.4992, 'train_samples_per_second': 12.218, 'train_steps_per_second': 1.529, 'total_flos': 2749300770078720.0, 'train_loss': 0.4632857378442489, 'epoch': 5.0})

In [20]:
from transformers import T5ForConditionalGeneration, T5Tokenizer

last_checkpoint = "./results/checkpoint-1500"
finetuned_model = T5ForConditionalGeneration.from_pretrained(last_checkpoint)
tokenizer = T5Tokenizer.from_pretrained(last_checkpoint)

test_data = dataset['test'] 

def generate_answer(question):
    inputs = tokenizer(question, return_tensors="pt")
    #outputs = finetuned_model.generate(**inputs)
    # Generate prediction with beam search to reduce repetition
    # outputs = finetuned_model.generate(**inputs, num_beams=5, max_new_tokens = 100, no_repeat_ngram_size=5)
    outputs = finetuned_model.generate(**inputs, max_new_tokens=100, repetition_penalty=3.0)
    answer = tokenizer.decode(outputs[0], skip_special_tokens=True)
    #answer = tokenizer.decode(outputs, skip_special_tokens=True)
    return answer

answers = []

for example in test_data:
    question = example["question"]
    print("Question:", question)
    answer = generate_answer(question)
    print("Answer:", answer)
    answers.append({"question": question, "answer": answer})


with open('test_answers.json', 'w', encoding='utf-8') as f:
    json.dump(answers, f, ensure_ascii=False, indent=4)

print("Answers saved to test_answers.json")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Question: List all sports teams Anthony Grant, also known as Anthony Paul Shaun Andrew Daure Grant, played for from 2010 to 2020.
Answer: England national under-21 football team (2010), Manchester City F.C. (2010, 2011, 2012), Tottenham Hotspur A.F.C. (2011), West Bromwich Albion Town F.C. (2012), Stoke City F.C. (2013), Fulham F.C. (2014), Burnley F.C. (2015), Crystal Palace FC (2016, 2017, 2018, 2019, 2020)
Question: List all positions Oleksandr Turchynov, also known as Oleksandr Valentynovych Turchynov, held from 2010 to 2020.
Answer: President of Ukraine (2010, 2011, 2012), Prime Minister of Ukraine (2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020)
Question: List all political parties Paul Murphy was a member of from 2010 to 2020.
Answer: Democratic Party (2010, 2011, 2012), independent politician (2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020)
Question: List all sports teams Andy Carroll, also known as Andrew Thomas Carroll, played for from 2010 to 2020.
Answer: Englan

Answer: Real Madrid CF (2010, 2011, 2012), Spain national under-21 football team (2010), FC Barcelona (2011, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020)
Question: List all political parties Rachel Carling-Jenkins, also known as Rachel Carling, was a member of from 2010 to 2018.
Answer: Conservative Party (2010, 2011, 2012), Liberal Democrats (2010), Labour Party of Australia (2012, 2013, 2014, 2015, 2016, 2017, 2018)
Question: List all coaches of CD Tondela from 2015 to 2020
Answer: Yuriy Stabak (2015, 2016, 2017, 2018, 2019, 2020), Igor Kostunica (2015)
Question: List all positions Brian Cowen, also known as Brian Bernard Cowen, held from 2010 to 2011.
Answer: Minister for Health (2010, 2011), Leader of the Opposition (2010)
Question: List all political parties Ovidiu Silaghi, also known as Ovidiu Ioan Silaghi, was a member of from 2010 to 2020.
Answer: Democratic Convergence of Catalonia (2010, 2011, 2012), National Liberal Party (2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2

Answer: Giovanni Stroppa (2012, 2013, 2014), Giorgi Dionisi (2014, 2015, 2016, 2017, 2018, 2019, 2020)
Question: List all coaches of Silkeborg IF, also known as Silkeborg Idrætsforening, from 2015 to 2020
Answer: Erik Srensen (2015, 2016, 2017, 2018, 2019, 2020), Jörg Huldholt (2020)
Question: List all chairpersons of Roman Catholic Archdiocese of Cardiff, also known as Archdiocese of Cardiff, from 2010 to 2020
Answer: Peter O'Neill (2010, 2011, 2012), Michael McKinsey (2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020)
Question: List all coaches of Qhubeka Assos, also known as Dimension Data, from 2015 to 2020
Answer: Yuriy Kostuna (2015, 2016, 2017, 2018, 2019, 2020), Nikos Shrevenovich (2015)
Question: List all employers Billy Hill and the Hillbillies worked for from 2010 to 2020.
Answer: University of Queensland (2010, 2011, 2012), Australian National College of Art (2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020)
Question: List all entities that owned Jæren Line from 2010

Answer: Peter van Hooghout (2013, 2014), Jan Schreiber (2014, 2015, 2016, 2017, 2018, 2019, 2020)
Question: List all employers Pedro J. Ramírez worked for from 2010 to 2020.
Answer: University of California, Berkeley (2010, 2011, 2012), Universidad Nacional de Investigaciones Sociales Catalunyas (2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020)
Question: List all positions Charles Falconer, Baron Falconer of Thoroton, also known as Charles Falconer, held from 2015 to 2020.
Answer: Lord Chancellor of the Exchequer (2015, 2016, 2017, 2018, 2019, 2020), President-elect of the House of Commons (2019, 2020)
Question: List all coaches of SSU Politehnica Timișoara, also known as SS Politehnica Timișoara, from 2010 to 2019
Answer: tefan Sánchez (2010, 2011, 2012), Ion Săpișanu (2010), Mihail Raț (2012), Marius Pescara (2013), Andrei Petrej (2014, 2015, 2016, 2017, 2018, 2019)
Question: List all entities that owned Portrait of Professor Benjamin H. Rand from 2010 to 2020.
Answer: Universit

...

Answer: José Mourinho (2010, 2011, 2012), Luis Enrique (2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020)
Question: List all chairpersons of Social Democrats, also known as Socialdemokratiet, from 2010 to 2020
Answer: Yuriy Svdr (2010, 2011, 2012), Igor Krypanovich (2010), Mikhail Städer (2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020)
Question: List all positions Jyrki Katainen, also known as Jyrki Tapani Katainen, held from 2010 to 2020.
Answer: Minister of Foreign Affairs (2010, 2011, 2012), Prime Minister of Finland (2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020)
Question: List all employers Kaywin Feldman worked for from 2010 to 2020.
Answer: University of California, Berkeley (2010, 2011, 2012), Harvard Medical School (2010), Stanford University (2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020)
Question: List all sports teams Emily Sonnett played for from 2012 to 2020.
Answer: England national under-21 football team (2012, 2013, 2014), Manchester City F.C. (

Answer: Kyivyn Yerev (2010, 2011, 2012), Ihor Tyatov (2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020)
Question: List all chairpersons of TOP 09, also known as Tradition Responsibility Prosperity, from 2010 to 2020
Answer: Yuriy Kostuna (2010, 2011, 2012), Igor Sánchez-Sanchez (2010), Alessandro González-Croas (2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020)
Question: List all positions Mike Gallagher, also known as Michael John Gallagher, held from 2015 to 2020.
Answer: Minister for Health (2015, 2016, 2017, 2018, 2019, 2020), Leader of the Opposition (2019, 2020)
Question: List all sports teams Javier Mascherano, also known as Javier Alejandro Mascherano, played for from 2010 to 2020.
Answer: Arsenal F.C. (2010, 2011, 2012), Manchester City A.F.C. (2010), Chelsea F.C. (2011), Real Madrid CF (2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020)
Question: List all employers SethBling worked for from 2010 to 2020.
Answer: University of California, Berkeley (2010, 2011, 2012)

Answer: University of Kentucky (2010, 2011, 2012), Texas A&M University (2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020)
Question: List all positions Mohamed Ghannouchi held from 2010 to 2011.
Answer: President of Egypt (2010, 2011), Prime Minister of Egypt (2010)
Question: List all sports teams Jamal Crawford, also known as Aaron Jamal Crawford, played for from 2017 to 2020.
Answer: Oklahoma City Thunder (2017, 2018, 2019, 2020), Houston Rockets (2020)
Question: List all sports teams Humam Tariq, also known as Humam Tariq Faraj Naoush, played for from 2010 to 2020.
Answer: Pakistan national cricket team (2010, 2011, 2012), Mumbai Indians (2010), Delhi Daredevils (2011, 2013), Rajasthan Royals (2013), Chennai Super Kings (2014), Kolkata Knight Riders (2015), Gujarat Lions (2016), Punjab United F.C. (2018), Kerala Blasters FC (2017), Sunrisers Hyderabad (2018, 2019, 2020)
Question: List all employers Julia Y Chan, also known as Julia Y. Chan, worked for from 2010 to 2020.
Answer: 

Answer: New Zealand national cricket team (2010, 2011, 2012), Sydney Sixers (2010), Brisbane Heat (2011, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020)
Question: List all sports teams Filipe Morais, also known as Filipe Alexandre Major Morais, played for from 2010 to 2020.
Answer: FC Barcelona (2010, 2011, 2012), Real Madrid CF (2010), Chelsea F.C. (2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020)
Question: List all political parties Marina Weisband was a member of from 2010 to 2020.
Answer: Democratic Party (2010, 2011, 2012), Liberal Democrats of Germany (2010), Alternative for the Union (2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020)
Question: List all coaches of Spain women's national association football team, also known as Spain women's national soccer team, from 2010 to 2020
Answer: Mauricio Pochettino (2010, 2011, 2012), José Márcito Garca (2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020)
Question: List all educational institutions Shawné Michaelain Holloway, 

Answer: José Luis Rodrguez Sánchez (2015, 2016, 2017, 2018, 2019, 2020), Juan Carlos A. González (2020)
Question: List all sports teams Jonny Howson, also known as Jonathan Mark Howson, played for from 2010 to 2020.
Answer: England national under-21 football team (2010), Manchester City F.C. (2010, 2011, 2012), Tottenham Hotspur A.F.C. (2011), West Bromwich Albion F.C. (2012), Stoke City F.C. (2013), Burnley F.C. (2014), Middlesbrough F.C. (2015), Blackpool F.C. (2016), Crystal Palace FC (2017, 2018, 2019, 2020)
Question: List all entities that owned MyAnimeList, also known as MyAnimeList, LLC, from 2010 to 2019.
Answer: Yahoo (2010, 2011, 2012), MyAnimeList (2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019)
Question: List all political parties Sarath Fonseka was a member of from 2010 to 2020.
Answer: Democratic Convergence of Catalonia (2010, 2011, 2012), The People of Freedom (2010), Catalan European Democratic Party (2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020)
Question: List all s